In [64]:
import numpy as np
import math
from math import e

In [27]:
# define Pauli matrices
I = np.matrix([[1,0],[0,1]])
X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,0-1j],[0+1j,0]])
Z = np.matrix([[1,0],[0,-1]])

In [28]:
# any linear combination of I,X,Y,Z
def generate_combination(i,x,y,z):
    return i*I+x*X+y*Y+z*Z

In [36]:
def check_unitary(A):
    Adag = A.getH()
    if not (np.array_equal(np.matmul(Adag,A),I) and np.array_equal(np.matmul(A,Adag),I)):
        return False
    return True

In [419]:
def generate_hamiltonian(alpha,beta,gamma,delta):
    term1 = e**((alpha-beta/2-delta/2)*1j)*math.cos(gamma/2)
    term2 = e**((alpha-beta/2+delta/2)*1j)*math.sin(gamma/2)*(-1)
    term3 = e**((alpha+beta/2-delta/2)*1j)*math.sin(gamma/2)
    term4 = e**((alpha+beta/2+delta/2)*1j)*math.cos(gamma/2)
    hamiltonian = np.matrix([[term1,term2],[term3,term4]])
    # we verify that the hamiltonian is unitary, and if it isn't we return the identity
    if check_unitary(hamiltonian):
        return hamiltonian
    else:
        return I

#below are other possible functions to generate unitary matrices, but they do not have as good success rates
'''
def generate_hamiltonian(alpha,beta,xi,zeta):
    first_mat = np.matrix([[math.cos(alpha),-math.sin(alpha)],[math.sin(alpha),math.cos(alpha)]])
    second_mat = np.matrix([[e**((xi)*1j),0],[0,e**((zeta)*1j)]])
    third_mat = np.matrix([[math.cos(beta),math.sin(beta)],[-math.sin(beta),math.cos(beta)]])
    return np.matmul(np.matmul(first_mat,second_mat),third_mat)

def generate_hamiltonian(x,y,z):
    return np.matmul(np.matmul(X**x,Y**y),Z**z)
'''

'\ndef generate_hamiltonian(alpha,beta,xi,zeta):\n    first_mat = np.matrix([[math.cos(alpha),-math.sin(alpha)],[math.sin(alpha),math.cos(alpha)]])\n    second_mat = np.matrix([[e**((xi)*1j),0],[0,e**((zeta)*1j)]])\n    third_mat = np.matrix([[math.cos(beta),math.sin(beta)],[-math.sin(beta),math.cos(beta)]])\n    return np.matmul(np.matmul(first_mat,second_mat),third_mat)\n\ndef generate_hamiltonian(x,y,z):\n    return np.matmul(np.matmul(X**x,Y**y),Z**z)\n'

In [420]:
# change values of alpha, beta, gamma, and delta to see what sort of unitary matrices we return
a=1
b=1
c=1
d=1
A = generate_hamiltonian(a,b,c,d)

In [421]:
# should always be true
check_unitary(A)

True

In [422]:
# if A is identity, maybe try different values of alpha, beta, gamma, and delta
A

matrix([[ 0.87758256+0.j        , -0.25903472-0.40342268j],
        [ 0.25903472+0.40342268j, -0.36520321+0.79798357j]])

In [423]:
# generate random 2 dimensional vector
vec = np.random.rand(2)

In [424]:
def get_magnitude(v):
    return math.sqrt(v[0]**2+v[1]**2)

In [425]:
# change vector to have norm of 1
vec = vec/get_magnitude(vec)

In [426]:
vec

array([0.281239  , 0.95963776])

In [427]:
# check first evolution of vector, compare to evolution matrix
evolve(vec,A)

array([-0.00176906-0.38713964j, -0.27761212+0.87923335j])

In [428]:
def evolve(v,A):
    return np.matmul(A,v).A1

In [429]:
def generate_evolution_matrix(v,A,n):
    evolution_matrix = np.array([v,evolve(v,A)])
    v = evolve(evolve(v,A),A)
    for i in range(0,n):
        evolution_matrix = np.vstack( (evolution_matrix,np.array([v])) )
        v = evolve(v,A)
    return evolution_matrix

In [430]:
generate_evolution_matrix(vec,A,48)

array([[ 0.281239  +0.j        ,  0.95963776+0.j        ],
       [-0.00176906-0.38713964j, -0.27761212+0.87923335j],
       [ 0.42506136-0.45550394j, -0.44450627-0.64362503j],
       [ 0.22851606-0.05369717j,  0.96980358-0.06616671j],
       [-0.07736425-0.42122498j, -0.22051918+0.87633075j],
       [ 0.34276031-0.50769736j, -0.46887154-0.63633187j],
       [ 0.16554377-0.09156088j,  0.97261921-0.1349953j ],
       [-0.16112399-0.43776041j, -0.1676603 +0.86850153j],
       [ 0.25240345-0.54150499j, -0.49695409-0.62936593j],
       [ 0.09633274-0.11170516j,  0.96754955-0.20515806j],
       [-0.24885441-0.43521887j, -0.11962152+0.85694032j],
       [ 0.158305  -0.55565975j, -0.5290229 -0.62154367j],
       [ 0.02521619-0.11321608j,  0.95435473-0.27523319j],
       [-0.33611703-0.41306985j, -0.07669604+0.84292132j],
       [ 0.06495007-0.54990787j, -0.56505196-0.61163642j],
       [-0.04338088-0.09619971j,  0.93310423-0.34377349j],
       [-0.41846272-0.37180932j, -0.03887507+0.82772913j